In [1]:
import os
import json
import geopandas as gpd
import numpy
import psycopg2
import random
import pandas as pd

In [90]:
#SETTING UP CLIENTS DF

df = pd.read_excel('tmp_api/responses.xls')

df = df.rename(columns={'¿Qué edad tienes?': 'Edad', '¿Tienes hijos?': 'Hijos', '¿Trabajas actualmente?': 'Trabajo', '¿Valoras en gran medida la existencia de comercios cerca de tu zona?': 'Comercios', '¿Valoras en gran medida la existencia de estaciones de transporte público cerca de tu zona?': 'Transporte Publico', '¿Valoras en gran medida la existencia de lugares de ocio cerca de tu zona?': 'Ocio','¿Valoras en gran medida la existencia de colegios cerca de tu zona?': 'Colegios', '¿Valoras en gran medida la existencia de zonas verdes cerca de tu zona?': 'Zonas verdes', '¿Valoras en gran medida la existencia de centros sanitarios cerca de tu zona?': 'Centros Sanitarios', '¿Valoras negativamente la contaminación en tu zona?': 'Contaminacion', '¿El exceso de ruido supone un problema para ti?': 'Ruido', '¿Cuánto valoras la limpieza del barrio?': 'Limpieza','Ante la posibilidad de adquirir un coche electrico, ¿valoras la existencia de puntos de recarga?': 'Puntos de recarga', 'De las comodidades anteriores ¿cuáles serían las 3 que más valoras?': 'Comodidades', '¿Cuánto estarías dispuesto a pagar por el alquiler de una casa que ofrezca todas las comodidades que buscas?': 'Alquiler'})
df = df.drop(['Marca temporal', 'Puntuación','Comercios', 'Trabajo', 'Estado Civil', 'Ocio'], axis=1)

#adding a column for id
df.insert(0, 'id_cliente', range(0, len(df)))
#print(df.columns)
#print(df['Comodidades'])

cliente_df = pd.DataFrame()
cliente_df['id_cliente'] = df['id_cliente']
cliente_df['Transporte Publico'] = df['Transporte Publico']
cliente_df['Colegios'] = df['Colegios']
cliente_df['Zonas verdes'] = df['Zonas verdes']
cliente_df['Centros Sanitarios'] = df['Centros Sanitarios']
cliente_df['Contaminacion'] = df['Contaminacion']
cliente_df['Ruido'] = df['Ruido']
cliente_df['Limpieza'] = df['Limpieza']
cliente_df['Puntos de recarga'] = df['Puntos de recarga']

top3 = cliente_df.iloc[:, 1:].apply(lambda s: s.abs().nlargest(3).index.tolist(), axis=1)

clients_df = pd.DataFrame()
clients_df['id_cliente'] = cliente_df['id_cliente']
clients_df = clients_df.assign(Recomendacion=top3)
clients_df[['Recomendacion 1','Recomedacion 2', 'Recomendacion 3']] = pd.DataFrame(clients_df.Recomendacion.tolist(), index= clients_df.index)
clients_df = clients_df.drop(['Recomendacion'], axis = 1)

clients_df

,id_cliente,Recomendacion 1,Recomedacion 2,Recomendacion 3
0,0,Transporte Publico,Centros Sanitarios,Colegios
1,1,Zonas verdes,Limpieza,Centros Sanitarios
2,2,Transporte Publico,Centros Sanitarios,Colegios
3,3,Transporte Publico,Zonas verdes,Centros Sanitarios
4,4,Transporte Publico,Centros Sanitarios,Limpieza
...,...,...,...,...
155,155,Transporte Publico,Colegios,Zonas verdes
156,156,Transporte Publico,Puntos de recarga,Limpieza
157,157,Colegios,Zonas verdes,Transporte Publico
158,158,Contaminacion,Ruido,Limpieza


In [37]:

# SETTING UP BARRIOS DF
path = '../datos/datos_out/barrios_caracteristicas.geojson'


#CARGAMOS LOS DATOS DE LOS BARRIOS
with open('../datos/datos_out/barrios_caracteristicas.geojson') as json_file:
    json_data = json.load(json_file)

barrios_json = []
for i in range(len(json_data['features'])):
    barrios_json.append(json_data['features'][i])           #Guardamos resto de campos de geojson


barrios_gpd = gpd.GeoDataFrame.from_features(barrios_json)
barrios_gpd.crs = 'epsg:4326' #Aseguramos que la proyección es la adecuada para coordenadas GPS

barrios_gpd = barrios_gpd[['object_id_barrio', '%_zona_verde', 'nivel_acustico', 'num_hospitales', 'num_colegios', 'num_chargestations', 'calidad_ambiental', 'num_contenedores', 'num_transporte']]

random_quality = ['Desfavorable', 'Regular', 'Razonablemente buena', 'Buena'] # Declaramos los niveles que obtenemos de los datos
for i in barrios_gpd['calidad_ambiental']:
    
    barrios_gpd['calidad_ambiental'] = random.choices(random_quality, k = len(barrios_gpd['calidad_ambiental'])) # Generamos datos random para la calidad ambiental por cada barrio
        
score = [1,2,3,4]
punt_ambiental = []

for i in barrios_gpd['calidad_ambiental']:
    
    if i == 'Desfavorable':
        punt_ambiental.append(score[0])

    elif i == 'Regular':
        punt_ambiental.append(score[1])

    elif i == 'Razonablemente buena':
        punt_ambiental.append(score[2])

    elif i == 'Buena':
        punt_ambiental.append(score[3])

barrios_gpd['punt_ambiental'] = punt_ambiental

barrios_gpd = barrios_gpd.drop_duplicates(['object_id_barrio'])
barrios_gpd



,object_id_barrio,%_zona_verde,nivel_acustico,num_hospitales,num_colegios,num_chargestations,calidad_ambiental,num_contenedores,num_transporte,punt_ambiental
0,62,0.118869,3.0,1.0,6.0,NaN,Buena,1104.0,66.0,4
6,63,0.037566,3.5,NaN,1.0,2.0,Regular,972.0,48.0,2
12,76,0.205871,3.5,NaN,5.0,NaN,Buena,238.0,10.0,4
13,601,NaN,3.5,NaN,NaN,NaN,Razonablemente buena,NaN,NaN,3
19,137,0.187377,3.5,NaN,10.0,NaN,Desfavorable,474.0,42.0,1
...,...,...,...,...,...,...,...,...,...,...
115,28,0.087284,4.0,1.0,11.0,NaN,Desfavorable,279.0,14.0,1
116,37,0.134529,4.0,NaN,2.0,2.0,Desfavorable,580.0,27.0,1
117,249,NaN,3.0,1.0,2.0,NaN,Buena,185.0,14.0,4
118,201,0.153242,3.0,1.0,9.0,NaN,Razonablemente buena,214.0,21.0,3


In [38]:
#SETTING DATAFRAMES PER 
verdes_df = barrios_gpd[['object_id_barrio', '%_zona_verde']].sort_values(['%_zona_verde'], axis = 0, ascending = False)
acustico_df = barrios_gpd[['object_id_barrio', 'nivel_acustico']].sort_values(['nivel_acustico'], axis = 0)
hospitales_df = barrios_gpd[['object_id_barrio', 'num_hospitales']].sort_values(['num_hospitales'], axis = 0, ascending = False)
colegios_df = barrios_gpd[['object_id_barrio', 'num_colegios']].sort_values(['num_colegios'], axis = 0, ascending = False)
chargestation_df = barrios_gpd[['object_id_barrio', 'num_chargestations']].sort_values(['num_chargestations'], axis = 0, ascending = False)
contaminacion_df = barrios_gpd[['object_id_barrio', 'punt_ambiental']].sort_values(['punt_ambiental'], axis = 0, ascending = False)
contenedores_df = barrios_gpd[['object_id_barrio', 'num_contenedores']].sort_values(['num_contenedores'], axis = 0, ascending = False)
transporte_df = barrios_gpd[['object_id_barrio', 'num_transporte']].sort_values(['num_transporte'], axis = 0, ascending = False)
acustico_df



,object_id_barrio,nivel_acustico
37,617,2.0
56,620,2.5
98,618,2.5
93,265,2.5
44,5,2.5
...,...,...
86,60,4.0
103,23,4.0
45,15,4.0
99,12,4.5


In [89]:
caract_df = pd.DataFrame()

for key, values in clients_df.iteritems():
     if key == 'Recomendacion 1':
          for i in values:
               if i == 'Transporte Publico':
                    caract_df['client_id'] = clients_df['id_cliente'][clients_df['Recomendacion 1'] == 'Transporte Publico']
                    print(caract_df)
        
          



     client_id
0            0
2            2
3            3
4            4
5            5
..         ...
150        150
151        151
152        152
155        155
156        156

[86 rows x 1 columns]
     client_id
0            0
2            2
3            3
4            4
5            5
..         ...
150        150
151        151
152        152
155        155
156        156

[86 rows x 1 columns]
     client_id
0            0
2            2
3            3
4            4
5            5
..         ...
150        150
151        151
152        152
155        155
156        156

[86 rows x 1 columns]
     client_id
0            0
2            2
3            3
4            4
5            5
..         ...
150        150
151        151
152        152
155        155
156        156

[86 rows x 1 columns]
     client_id
0            0
2            2
3            3
4            4
5            5
..         ...
150        150
151        151
152        152
155        155
156        156

[86 row

C:\Users\Usuario\AppData\Local\Temp\ipykernel_149572\3649601290.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for key, values in clients_df.iteritems():


1 2 3 3 4 3 1 2 1 3 3 2 1 1 3 3 3 1 2 1 3 1 2 3 3 2 3 2 2 1 3 4 3 3 4 2 1 1 4 1 3 3 1 2 1 2 3 2 2 4 4 1 1 2 2 3 4 2 3 3 4 3 1 3 2 1 3 1 1 3 2 3 3 1 1 1 3 4 2 3 2 2 3 2 2 4 1 4 3 2 4 4 3 3 3 3 3 1 4 2 1 3 4 3 2 3 1 4 3 1 1 3 4 4 4 4 3 3 2 2
        calidad_ambiental  punt_ambiental
0            Desfavorable               1
1                 Regular               2
2    Razonablemente buena               3
3    Razonablemente buena               3
4                   Buena               4
..                    ...             ...
115                 Buena               4
116  Razonablemente buena               3
117  Razonablemente buena               3
118               Regular               2
119               Regular               2

[120 rows x 2 columns]
